In [ ]:
! pip install -q sentence-transformers datasets pylate beir ranx

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 138.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00

In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
! rm -r output/
! rm -r pylate-index/

rm: cannot remove 'output/': No such file or directory
rm: cannot remove 'pylate-index/': No such file or directory


In [ ]:
import torch
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)

from pylate import evaluation, losses, models, utils

# Define model parameters for contrastive training
model_name = "rasyosef/roberta-medium-amharic"  # Choose the pre-trained model you want to use as base
batch_size = 32  # Larger batch size often improves results, but requires more memory

num_train_epochs = 4  # Adjust based on your requirements
# Set the run name for logging and output directory
run_name = "colbert-medium-amharic"
output_dir = f"output/{run_name}"

# 1. Here we define our ColBERT model. If not a ColBERT model, will add a linear layer to the base encoder.
model = models.ColBERT(
  model_name_or_path=model_name,
  document_length=256
)

# Compiling the model makes the training faster
# model = torch.compile(model)

In [ ]:
model.query_length, model.document_length

(32, 256)

In [ ]:
# Load Dataset
from datasets import load_dataset

am_relevance_dataset = load_dataset("yosefw/amharic-news-retrieval-dataset-v2-with-negatives-V2")#.select(range(4_000))
am_relevance_dataset

In [ ]:
test_passage_ids = set(am_relevance_dataset["test"]["passage_id"])
len(test_passage_ids)

6764

In [ ]:
from datasets import Dataset
from tqdm import tqdm
import random

ds_rows = []
for row in tqdm(am_relevance_dataset["train"]):
  neg_passages = row["negative_passages"]
  # neg_passages = list(filter(lambda x: x["passage_id"] not in test_passage_ids, neg_passages))
  neg_passages_filtered = neg_passages[:4] + neg_passages[-4:]

  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "positive": row["passage"],
      "negative_1": neg_passages_filtered[0]["passage"],
      "negative_2": neg_passages_filtered[2]["passage"],
      "negative_3": neg_passages_filtered[4]["passage"],
      "negative_4": neg_passages_filtered[6]["passage"],
    })

  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "positive": row["passage"],
      "negative_1": neg_passages_filtered[1]["passage"],
      "negative_2": neg_passages_filtered[3]["passage"],
      "negative_3": neg_passages_filtered[5]["passage"],
      "negative_4": neg_passages_filtered[7]["passage"],
    })
  # print(ds_rows)
  # break

relevance_dataset = Dataset.from_list(ds_rows).sort("query_id")#.select(range(4000))
relevance_dataset

100%|██████████| 61469/61469 [00:25<00:00, 2383.06it/s]


Dataset({
    features: ['query_id', 'passage_id', 'query', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4'],
    num_rows: 122938
})

In [ ]:
len(test_passage_ids.intersection(relevance_dataset["passage_id"]))

161

In [ ]:
# Split the dataset (this dataset does not have a validation set, so we split the training set)

EVAL_SIZE = 4_000

eval_dataset = relevance_dataset.select(range(EVAL_SIZE))
train_dataset = relevance_dataset.select(range(EVAL_SIZE, len(relevance_dataset)))

train_dataset[0], train_dataset

In [ ]:
eval_dataset[0], eval_dataset

In [ ]:
# Define the loss function
train_loss = losses.Contrastive(model=model)

# Initialize the evaluator
dev_evaluator = evaluation.ColBERTTripletEvaluator(
    anchors=eval_dataset["query"] * 4,
    positives=eval_dataset["positive"] * 4,
    negatives=eval_dataset["negative_1"] + eval_dataset["negative_2"] + eval_dataset["negative_3"] + eval_dataset["negative_4"],
)

In [ ]:
eval_steps = 1000

# Configure the training arguments (e.g., batch size, evaluation strategy, logging steps)
args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    run_name=run_name,  # Will be used in W&B if `wandb` is installed
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    eval_strategy="steps",
    eval_steps=eval_steps,
    save_strategy="steps",
    save_steps=eval_steps,
    # save_total_limit=2,
    logging_strategy="steps",
    logging_steps=eval_steps,
)

# Initialize the trainer for the contrastive training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
    data_collator=utils.ColBERTCollator(model.tokenize),
)

In [ ]:
# Start the training process
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
1000,0.463200,0.290327,0.968125
2000,0.321200,0.259508,0.971000
3000,0.266100,0.240049,0.973313


Step,Training Loss,Validation Loss,Accuracy
1000,0.463200,0.290327,0.968125
2000,0.321200,0.259508,0.971000
3000,0.266100,0.240049,0.973313
4000,0.226800,0.217545,0.976313
5000,0.171200,0.204488,0.978750
6000,0.158300,0.199975,0.979188
7000,0.146300,0.194563,0.980188
8000,0.128300,0.195501,0.980438
9000,0.107700,0.192818,0.980938
10000,0.106900,0.189652,0.981063


TrainOutput(global_step=14868, training_loss=0.1691610871510766, metrics={'train_runtime': 8042.453, 'train_samples_per_second': 59.155, 'train_steps_per_second': 1.849, 'total_flos': 0.0, 'train_loss': 0.1691610871510766, 'epoch': 4.0})

### **Evaluate**

In [ ]:
dev_evaluator(
    model=model,
    # model=models.ColBERT("/content/output/contrastive-bert-small/checkpoint-2850"),
    output_path="."
  )

{'accuracy': 0.9816875457763672}

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_WRITE")

# push model to hub
# trainer.model.push_to_hub("ColBERT-Amharic-Medium")

### **BIER Eval**

In [ ]:
from pylate import evaluation, indexes, models, retrieve

# Step 1: Initialize the ColBERT model

# dataset = "scifact" # Choose the dataset you want to evaluate
model = models.ColBERT(
    model_name_or_path="rasyosef/ColBERT-Amharic-Medium",
    device="cuda" # "cpu" or "cuda" or "mps"
)

# Step 2: Create a Voyager index
index = indexes.Voyager(
    index_folder="pylate-index",
    index_name="colbert-eval",
    override=True,  # Overwrite any existing index
)

test_dataset = am_relevance_dataset["test"]

documents = [
    {"id": pid, "text": pos}
    for pid, pos in dict(zip(
        am_relevance_dataset["test"]["passage_id"] + am_relevance_dataset["train"]["passage_id"],
        am_relevance_dataset["test"]["passage"] + am_relevance_dataset["train"]["passage"]
    )).items()
  ]

queries = [q for q in test_dataset["query"]]
qrels = {query: {pid:1} for query, pid in zip(test_dataset["query"], test_dataset["passage_id"])}

# Step 4: Encode the documents
documents_embeddings = model.encode(
    [document["text"] for document in documents],
    batch_size=32,
    is_query=False,  # Indicate that these are documents
    show_progress_bar=True,
)

# Step 5: Add document embeddings to the index
index.add_documents(
    documents_ids=[document["id"] for document in documents],
    documents_embeddings=documents_embeddings,
)

# Step 6: Encode the queries
queries_embeddings = model.encode(
    queries,
    batch_size=32,
    is_query=True,  # Indicate that these are queries
    show_progress_bar=True,
)

# Step 7: Retrieve top-k documents
retriever = retrieve.ColBERT(index=index)
scores = retriever.retrieve(
    queries_embeddings=queries_embeddings,
    k=100,  # Retrieve the top 100 matches for each query
    batch_size=128,
)

# Step 8: Evaluate the retrieval results
results = evaluation.evaluate(
    scores=scores,
    qrels=qrels,
    queries=queries,
    metrics=[f"ndcg@{k}" for k in [5, 10, 100]] # NDCG for different k values
    + ["recall@5", "recall@10", "recall@50", "recall@100"]   # Recall at k
    + ["mrr@5", "mrr@10", "mrr@100"]
)

print(results)

In [ ]:
# Step 8: Evaluate the retrieval results
results = evaluation.evaluate(
    scores=scores,
    qrels=qrels,
    queries=queries,
    metrics=[f"ndcg@{k}" for k in [5, 10, 100]] # NDCG for different k values
    + ["recall@5", "recall@10", "recall@50", "recall@100"]   # Recall at k
    + ["mrr@5", "mrr@10", "mrr@100"]
)

print(results)

metrics = ["ndcg@5", "ndcg@10", "ndcg@100", "mrr@5", "mrr@10", "mrr@100", "recall@5", "recall@10", "recall@50", "recall@100"]
for m in metrics:
  print(f"{m}: {results[m]}")

{'ndcg@5': np.float64(0.8008276603008163), 'ndcg@10': np.float64(0.8111646117466479), 'ndcg@100': np.float64(0.8228976121185352), 'recall@5': np.float64(0.8819566490919742), 'recall@10': np.float64(0.9137375512595196), 'recall@50': np.float64(0.9565026362038664), 'recall@100': np.float64(0.9685120093731693), 'mrr@5': np.float64(0.7733401679359501), 'mrr@10': np.float64(0.7776421899554589), 'mrr@100': np.float64(0.7800789457297966)}
ndcg@5: 0.8008276603008163
ndcg@10: 0.8111646117466479
ndcg@100: 0.8228976121185352
mrr@5: 0.7733401679359501
mrr@10: 0.7776421899554589
mrr@100: 0.7800789457297966
recall@5: 0.8819566490919742
recall@10: 0.9137375512595196
recall@50: 0.9565026362038664
recall@100: 0.9685120093731693
